In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score

import string, nltk, re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

**Let us use the datasets from Kaggle.**

In [ ]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
train.isnull().sum()

In [ ]:
train = train.drop(columns = ['keyword','location'])
train.dtypes

In [ ]:
train # Want to see whether the columns are dropped.

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
test.info()

In [ ]:
test.sample(10)

In [ ]:
test.isnull().sum()

In [ ]:
train['target'].value_counts()

In [ ]:
print(f"Tweets that does not contain information about disaster :\n\n {train.loc[train['target']==0,'text'][20:25].values}", end="\n")

print()

print(f"Tweets that contains information about disaster :\n\n {train.loc[train['target']==1,'text'][20:25].values}", end="\n")

In [ ]:
def preprocessing(text):
    text = text.lower()
    pattern = re.compile('[^a-z]')
    words = nltk.word_tokenize(text)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    words = [PorterStemmer().stem(word) for word in words if word.lower() not in stop_words]
    preprocessed_text = ' '.join(words)
    return preprocessed_text

In [ ]:
train['text_cleaned']=train['text'].apply(preprocessing)
test['text']=test['text'].apply(preprocessing)
train

In [ ]:
x=train['text_cleaned'].values
y=train['target'].values

In [ ]:
classifier=TfidfVectorizer()
x=classifier.fit_transform(x)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.10,random_state=27,stratify=y)

In [ ]:
logReg=LogisticRegression(penalty='l2')
logReg.fit(x_train,y_train)
pred=logReg.predict(x_test)

In [ ]:
R=logReg.predict(x_train)
accuracy_score(y_train,R)

In [ ]:
accuracy_score(y_test,pred)

In [ ]:
from sklearn.svm import SVC
svc_model=SVC()
svc_model.fit(x_train,y_train)

In [ ]:
y_pre=svc_model.predict(x_test)

In [ ]:
score=accuracy_score(y_test,y_pre,normalize=True)
print(score)

In [ ]:
x=classifier.transform(test['text'])

In [ ]:
predictions=svc_model.predict(x)

In [ ]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score, f1_score
from sklearn.metrics import accuracy_score
print("Mean Square Error is: ", mse(y_test, pred))
print("F1 Score is: ", f1_score(y_test, pred))
print("R Square score is: ", r2_score(y_test, pred))
print("Accuracy score is ", accuracy_score(y_test, pred))
print(f"Accuracy score is {((accuracy_score(y_test, pred))*100).round(2)}%")

In [ ]:
submission = pd.DataFrame({'id':test['id'],'target':predictions})
submission.to_csv('submission.csv',index=False)